### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [4]:
total_players = purchase_data['SN'].unique()
datacount = purchase_data['SN'].count()
print(f'Unique players {len(total_players)} out of {datacount} entries')

Unique players 576 out of 780 entries


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
unique_items = purchase_data['Item Name'].unique()
print(f'Number of Unique Items is {len(unique_items)}')

Number of Unique Items is 179


In [6]:
average_price = purchase_data['Price'].mean()
total_revenue = purchase_data['Price'].sum()
print(f'Average price is ${average_price:0.2f} for a total revenue of ${total_revenue:.2f}')

Average price is $3.05 for a total revenue of $2379.77


In [7]:
total_df = pd.DataFrame({'Unique Items' : [f'{len(unique_items)}'],
                        'Average Price' : [f'${average_price:0.2f}'],
                        'Total Revenue' : [f'${total_revenue:.2f}']})
total_df.index = ['Summary']


In [8]:
total_df

,Unique Items,Average Price,Total Revenue
Summary,179,$3.05,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [9]:
byID = purchase_data.groupby("SN")
player_count = len(byID)
demographics = purchase_data.groupby(['SN', 'Gender']).size().unstack().count()
df = pd.DataFrame(demographics)
df.columns = ['Count']
df['Percent'] = df['Count'] / player_count * 100

In [10]:
df

,Count,Percent
Gender,,
Female,81,14.062500
Male,484,84.027778
Other / Non-Disclosed,11,1.909722



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [11]:
purch_count = purchase_data.groupby(['SN', 'Gender']) ['Price'].agg(['sum','count'])
df = purch_count.reset_index()


In [12]:
byGender = df.groupby('Gender').agg({'sum': ['count', 'sum', 'mean'],
                         'count': ['mean','sum']})
byGender.columns = ['Number of Buyers', 
                    'Purchase Total', 
                    'Average Purchase per Person', 
                    'Avg. Purchases Per Person',
                   'Total Items']

byGender['Average Purchase Price'] = byGender['Purchase Total'] / byGender['Total Items']

In [13]:
byGender_df = byGender[['Number of Buyers','Total Items', 'Average Purchase Price', 'Average Purchase per Person']]

In [14]:
byGender_df

,Number of Buyers,Total Items,Average Purchase Price,Average Purchase per Person
Gender,,,,
Female,81,113,3.203009,4.468395
Male,484,652,3.017853,4.065372
Other / Non-Disclosed,11,15,3.346000,4.562727


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [15]:
purchase_data['Age'].describe()

count    780.000000
mean      22.714103
std        6.659444
min        7.000000
25%       20.000000
50%       22.000000
75%       25.000000
max       45.000000
Name: Age, dtype: float64

In [16]:
ages = [0, 10 ,20 ,30 , 40, 50]
age_labels = ['Child', 'Teen', 'Young Adult', 'Adult', 'Older']

In [17]:
by_age_df = purchase_data
by_age_df['Age Category'] = pd.cut(purchase_data['Age'], ages, labels = age_labels)

In [73]:
by_age_df_2 = by_age_df.groupby(['SN','Age Category'])['Price'].agg(['sum','count'])
df = by_age_df_2.reset_index()
df = df[df['count']>0]
df['person'] = 1
df = df.groupby('Age Category').agg({'count':'count'}).reset_index()
df['Percent'] = df['count']/ player_count * 100

df['Percent'] = df['Percent'].map('{:0.2f}%'.format)
df = df.rename(columns={'count':'Number of Buyers'})

In [74]:
df

,Age Category,Number of Buyers,Percent
0,Child,24,4.17%
1,Teen,191,33.16%
2,Young Adult,291,50.52%
3,Adult,63,10.94%
4,Older,7,1.22%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [75]:
age_purch_df = by_age_df.groupby(['SN', 'Age Category']) ['Price'].agg(['sum','count'])
df = age_purch_df.reset_index()
df = df[df['count']>0]

byAge = df.groupby('Age Category').agg({'sum': ['count', 'sum', 'mean'],
                        'count': ['mean','sum']})
byAge.columns = ['Player Count', 
                    'Purchase Total', 
                    'Average Purchase per Person', 
                    'Avg. Purchases Per Person',
                   'Total Items']

byAge['Average Purchase Price'] = byAge['Purchase Total'] / byAge['Total Items']

byAge = byAge[['Player Count','Total Items', 'Average Purchase Price', 'Average Purchase per Person']]
byAge['Average Purchase Price'] = byAge['Average Purchase Price'].map('${:0.2f}'.format)
byAge['Average Purchase per Person'] = byAge['Average Purchase per Person'].map('${:0.2f}'.format)

In [76]:
byAge

,Player Count,Total Items,Average Purchase Price,Average Purchase per Person
Age Category,,,,
Child,24,32,$3.41,$4.54
Teen,191,254,$3.06,$4.07
Young Adult,291,402,$2.99,$4.13
Adult,63,85,$3.15,$4.25
Older,7,7,$3.08,$3.08


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [77]:
top_df = purchase_data.groupby(['SN'])['Price'].agg(['count', 'sum'])
top_df = top_df.reset_index()
top_df.rename(columns={'SN' : 'Player ID',
                         'count' : 'Items',
                        'sum': 'Purchase Total'
                        }, inplace = True)

top_df.sort_values('Purchase Total',ascending = False, inplace = True)
top_df = top_df.set_index('Player ID')
top_df['Purchase Total'] = top_df['Purchase Total'].map('${:0.2f}'.format)
top_df.head(10)

,Items,Purchase Total
Player ID,,
Lisosia93,5,$18.96
Idastidru52,4,$15.45
Chamjask73,3,$13.83
Iral74,4,$13.62
Iskadarya95,3,$13.10
Ilarin91,3,$12.70
Ialallo29,3,$11.84
Tyidaim51,3,$11.83
Lassilsala30,3,$11.51


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [115]:
items_df = purchase_data[['Item ID', 'Item Name', 'Price']]

lookup = items_df.drop_duplicates(subset=['Item ID'])

df = items_df.groupby('Item ID',as_index=False,group_keys=False)['Price'].agg(['sum', 'count'])
df1 = df.reset_index()
df2 = pd.merge(df1, lookup, on='Item ID')

sum_s = df2['sum']

df2['Price'] = df2['Price'].map('${:0.2f}'.format)
df2['sum'] = df2['sum'].map('${:0.2f}'.format)
df3 = df2.sort_values('count', ascending = False)

df3 = df3[['Item ID','Item Name', 'count', 'Price', 'sum']]
df3 = df3.rename(columns = {'Item ID': 'ID',
                      'Item Name':'Name', 
                      'count': 'Quantity', 
                      'Price': 'Unit Price', 
                      'sum': 'Total'})

In [116]:



df3 

,ID,Name,Quantity,Unit Price,Total
90,92,Final Critic,13,$4.88,$59.99
174,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
141,145,Fiery Glass Crusader,9,$4.58,$41.22
129,132,Persuasion,9,$3.19,$28.99
105,108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
...,...,...,...,...,...
40,42,The Decapitator,1,$1.75,$1.75
49,51,Endbringer,1,$4.66,$4.66
115,118,"Ghost Reaver, Longsword of Magic",1,$2.17,$2.17
101,104,Gladiator's Glaive,1,$1.93,$1.93


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [119]:

df3 = df2.sort_values(sum_s, ascending = [False]*len(sum_s))

df3 = df2[['Item ID','Item Name', 'count', 'Price', 'sum']]


df3 = df3.rename(columns = {'Item ID': 'ID',
                      'Item Name':'Name', 
                      'count': 'Quantity', 
                      'Price': 'Unit Price', 
                      'sum': 'Total'})




ValueError: Length of ascending (179) != length of by (1)

In [113]:
df3

,ID,Name,Quantity,Unit Price,Total
61,63,Stormfury Mace,2,$4.99,$9.98
28,29,"Chaos, Ender of the End",5,$1.98,$9.90
169,173,Stormfury Longsword,2,$4.93,$9.86
36,38,"The Void, Vengeance of Dark Magic",4,$2.37,$9.48
139,143,Frenzied Scimitar,6,$1.56,$9.36
...,...,...,...,...,...
103,106,Crying Steel Sickle,3,$3.41,$10.23
147,151,Severance,3,$3.40,$10.20
107,110,Suspension,7,$1.44,$10.08
101,104,Gladiator's Glaive,1,$1.93,$1.93


In [121]:
sum_s

0       5.12
1      11.77
2      14.88
3      14.94
4       8.50
       ...  
174    50.76
175    26.88
176     8.30
177    12.09
178     3.27
Name: sum, Length: 179, dtype: float64